# Assignment 2 - Microbial GWAS

*Developed by Jimmy Liu, Jun Duan and William Hsiao*

## Learning Objectives
* Gain familiarity with command line tools to run microbial GWAS
* Understand the key analytical steps of GWAS
* Understand how to interpret GWAS output
* Identify the limitations of the presented methods

## Background
This assignment will focus on *Salmonella enterica*, an enteric pathogen that primarily spreads by human consumption of contaminated foods in developed countries, such as Canada and USA. You will examine isolates of *Salmonella* serovar Heidelberg from three epidemiologically distinct foodborne outbreaks that occurred in Quebec, Canada between 2012-2014. More detailed background on these foodborne outbreaks are described in the assigned pre-reading published by [Bekal et al. (2014)](https://pubmed.ncbi.nlm.nih.gov/26582830/).

As you have learned in previous lectures, phylogenetic methods can be applied to infer the relationships of these outbreak isolates and identify clonal strains that share a common origin. Outbreak tracing will however not be the purpose here, and instead you are provided with the epidemiology investigation results to identify genetic features that can distinguish *Salmonella* isolates of different outbreak origins. 

For example, Outbreak 1 isolates may carry a unique gene (Gene A) that is absent in the isolates from all other outbreaks. The presence and absence of Gene A would thus be a strongly predictive feature of outbreak origin. 

Throughout the assignment, you will be provided with detailed instructions on how to conduct a genome-wide survey of the bacterial genomes to identify all the genes unique to each outbreak. While this exercise focuses on the genetic association to each outbreak, the same approach can be readily extended to identifying genetic association of other phenotypes such as virulence, antimicrobial susceptibility, and transmissibility.

## Getting Started
* *Salmonella* genomes (*N* = 46) are in the shared directory: `/opt/share/gwas/genomes/`
* Metadata of the outbreak dataset is in the shared directory: `/opt/share/gwas/outbreak_metadata.csv`
* Pre-computed analysis results are in the shared directory: `/opt/share/gwas/precomputed_results/`

Let's begin by copying all the data from the shared directory to our current directory using the command `cp`

In [ ]:
# -r option indicates recursive copy and . refers to the current directory
cp -r /opt/share/gwas/* .

Use the `ls` command to verify that all the folders and files have been copied to our current directory.

In [ ]:
ls

To have the bioinformatics tools and their dependencies available in our analysis environment, use the `conda` command to activate the environment. All of the tools required to complete this assignment have been packaged in a conda environment called `gwas`.

In [ ]:
conda activate gwas

## Pan-Genome Analysis with Prokka and Roary

Prior to testing for genetic association, you first need to obtain information on what genetic features are present in each genome. You will run a gene prediction tool called `Prokka` and a pan-genome pipeline called `Roary` to conduct genome annotation and compute the pan-genome of the dataset. `Roary` generates a multitude of outputs including core genome alignments, genome annotations, etc. For our purpose, the key output from `Roary` is the gene presence/absence matrix (`gene_presence_absence.csv`) in which the columns are the individual genomes and the rows are the predicted genes. Depending on the tools used to prepare the genotype matrix, different tools can represent presence and absence in different ways. A typical representation is using 0 for absence and 1 for presence.

In consideration of time, the `Prokka` results have been pre-computed, and they can be found under `precomputed_results/prokka`. The primary `Prokka` output of interest is GFF files. GFF (General Feature Format) is a standard file format that encodes feature annotations for nucleic acid sequences. The information is formatted as a tab-delimited table with each row corresponding to a unique genetic feature (e.g. Coding or non-coding sequence) and the columns encode contextual information about the features such as its genomic position, +/- strand, gene name, encoded protein product, etc.

In [ ]:
# Print the content of the GFF file for Sample SH12-001
# head and tail are used in combination to skip the contig information in the file
head -n 40 precomputed_results/prokka/SH12-001.gff | tail -n 3

Expected outcome:

![img](https://github.com/jimmyliu1326/MBB_mGWAS/raw/main/img/gff_output.png)

<br>

> **CHECKPOINT**: 
> What is the protein product encoded by the first gene in the GFF file?

With the genome annotations pre-computed, you will run `Roary` on the GFF files of the entire dataset. In brief, `Roary` will analyze each GFF file given and aggregate all the features to construct the bacterial pan-genome and report gene presence and absence in a comma delimited (csv) file.

A typical `Roary` command looks like: 
```
roary [options] [path to GFF files]
```

`Roary` options explained:
* `-p` option specifies the number of compute cores to use for process parallelization
* `-f` option specifies the path to store the outputs

In [ ]:
roary -p 8 -f results/roary precomputed_results/prokka/*.gff

If the above command ran to completion successfully, you should now find a list of output files generated under `results/roary/`. `Roary` encodes the gene presence and absence information in two different formats. There is the data-rich format called `gene_presence_absence.csv` and a reduced format called `gene_presence_absence.Rtab`. 

There are two key differences between the two formats. First, the data-rich format contains additional information about gene annotation (e.g. encoded protein product) and summary statistics of genotype frequency. These additional information are encoded from columns 2 to 14 in `gene_presence_absence.csv`. Second, the data-rich format encodes gene presence using gene identifiers and gene absence using empty values. Conversely, the reduced format (`gene_presence_absence.Rtab`) represents gene presence and absence using numerical values (0 = absence; 1 = presence).

Let's take a glance at both formats to appreciate the differences and similarities.

In [ ]:
# Print content of the data-rich csv format
# As the file contains >3000 rows, use head cmd to only print the first N rows
# Skip columns 2 ~ 14 in the file using csvcut cmd
# Print a pretty table using column cmd
head -n 4 results/roary/gene_presence_absence.csv | csvcut -c 1,15-22 | column -t -s,

Expected outcome:

![img](https://github.com/jimmyliu1326/MBB_mGWAS/raw/main/img/gene_pres_abs_csv.png)

In [ ]:
# Print content of the reduced Rtab format
# As the file contains >3000 rows, use head cmd to only print the first N rows
# Parse tab-delimited columns using cut cmd
# Print a pretty table using column cmd
head -n 4 results/roary/gene_presence_absence.Rtab | cut -f 1-9 | column -t

Expected outcome:

![img](https://github.com/jimmyliu1326/MBB_mGWAS/raw/main/img/gene_pres_abs_tab.png)

> **CHECKPOINT:**
> What are the column headers referring to?

## Test Genome-wide association with Scoary

`Scoary` is designed to perform statistical tests (Fischer's exact test) on the genetic features summarized by `Roary` (notice the similarity in the tool names). In order to conduct GWAS with `Scoary`, the pre-requisites are genotype and phenotype matrices. By this stage, you will have prepared the genotype matrix using `Roary`. Specifically, `Scoary` prefers to process the data-rich format (`results/roary/gene_presence_absence.csv`) as the input genotype matrix.

For the phenotype matrix, it requires a .csv file with rows as individual samples (bacterial isolates) and columns as different phenotypic attributes. You can include as many phenotypes as you like in the matrix, and `Scoary` will write the results for each phenotype to a different output file. Because `Scoary` uses Fischer's exact tests to evaluate association, each column must be a **binary** variable. Therefore, you can't simply have the three *Salmonella* outbreak identifiers in a single column. Instead, multilevel phenotypes have to be encoded using a series of columns. As such, the outbreak information have to be encoded as three columns. Each column will represent a distinct outbreak and carry a value of `0` or `1` to indicate whether a given sample belongs to the outbreak in question (0 = No; 1 = Yes).

The phenotype matrix has already been formatted accordingly and can be found in your current directory called `outbreak_metadata.csv`. Use the command `head` to print the first few lines of the matrix.

In [ ]:
# use the -n option to print a specific number of lines (by default head prints the first 10 lines)
head -n 5 outbreak_metadata.csv

To calculate the statistical significance of association, you will run 1000 permutations of the phenotypic labels to construct a null distribution of test statistics and calculate the p-value (probability of obtaining a test statistic at least as extreme as the observed test statistic under the null). Lastly, the Benjamini-Hochberg method will be used to correct for false discovery rate.

A typical `Scoary` command looks like:
```
scoary [options] -t [path to phenotype matrix] -g [path to genotype matrix]
```

`Scoary` options explained:
* `--threads` specifies the number of compute cores to use for process parallelization
* `--no-time` to prevent appending time information to the output file name
* `-p` specifies a p-value cutoff to filter the results by
* `-o` specifies the path to store the outputs
* `-c` specifies the method to correct p-values for multiple testing
* `--permute` specifies the number of rounds of permutations to run

In [ ]:
scoary --threads 8 --no-time -o results/scoary \
       -p 0.05 -c BH --permute 1000 \
       -t outbreak_metadata.csv \
       -g results/roary/gene_presence_absence.csv

## Interpreting GWAS results

The GWAS results generated by `Scoary` are written to `results/scoary`. In the output directory, you should expect to find three csv files corresponding to the three categories of our phenotype of interest. The results are formatted as tabular data that can be directly viewed in JupyterHub by double clicking on the files or you can choose to download the files and open them in Excel. 

In order to interpret the GWAS results, we must first understand the underlying hypothesis that is being tested for. When testing the statistical association of a gene, we are evaluating whether there is a disproportion of gene presence and absence among two populations. In other words, we are testing a null hypothesis that a gene is found equally frequent in two populations. Visually, a 2x2 contingency table can be drawn to compare gene frequency in two populations:

|  | Lineage 1 (Positive) | Not Lineage 1 (Negative) | Total |
|---|---|---|---|
| Carries gene A | 5 | 20 | 25 |
| Does not carry gene A | 20 | 5 | 25 |
| | 25 | 25 | 50 |

This method of hypothesis testing is repeated for all of the genes found in the pan-genome of the dataset, and hence why the Benjamini-Hochberg procedure is applied to adjust the p-values for multiple testing.

Below you are provided a [table created by the Scoary developer](https://github.com/AdmiralenOla/Scoary/blob/master/README.md#output) to help interpret each column in the `Scoary` results:

| Column name | Explanation |
| ----------- | ----------- |
| Gene | The gene name |
| Non-unique gene name | The non-unique gene name |
| Annotation | Annotation |
| Number_pos_present_in | The number of trait-positive isolates this gene was found in |
| Number_neg_present_in | The number of trait-negative isolates this gene was found in |
| Number_pos_not_present_in | The number of trait-positive isolates this gene was not found in |
| Number_neg_not_present_in | The number of trait-negative isolates this gene was not found in |
| Sensitivity | The sensitivity if using the presence of this gene as a diagnostic test to determine trait-positivity |
| Specificity | The specificity if using the non-presence of this gene as a diagnostic test to determine trait-negativity |
| Odds_ratio | [Odds ratio](https://en.wikipedia.org/wiki/Odds_ratio) |
| p_value | The naïve p-value for the null hypothesis that the presence/absence of this gene is unrelated to the trait status |
| Bonferroni_p | A p-value adjusted with Bonferroni's method for multiple comparisons correction (An [FWER](https://en.wikipedia.org/wiki/Familywise_error_rate) type correction) |
| Benjamini_H_p | A p-value adjusted with Benjamini-Hochberg's method for multiple comparisons correction (An [FDR](https://en.wikipedia.org/wiki/False_discovery_rate) type correction) |
| Max_pairwise_comparisons | The maximum number of pairs that contrast in both gene and trait characters that can be drawn on the phylogenetic tree without intersecting lines (Read & Nee, 1995; Maddison, 2000) |
| Max_supporting_pairs | The maximum number of these pairs (Max_pairwise_comparisons) that support A->B or A->b, depending on the odds ratio. |
| Max_opposing_pairs | The maximum number of these pairs (Max_pairwise_comparisons) that oppose A->B or A->b, depending on the odds ratio. |
| Best_pairwise_comp_p | The p-value corresponding to the highest possible number of supporting pairs and the lowest possible number of opposing pairs, e.g. the lowest p-value you could end up with when picking a set of maximum number of pairs. |
| Worst_pairwise_comp_p | The p-value corresponding to the lowest possible number of supporting pairs and the highest possible number of opposing pairs, e.g. the highest p-value you could end up with when picking a set of maximum number of pairs. |
| Empirical_p | Empirical p-value after permutations and ranking of all test estimators. The test estimator used is number of successes (ie. AB-ab supporting pairs) divided by the number of trials (ie. the maximum number of contrasting pairs). This test estimator seem to approach a normal distribution. Empirical p is calculated by (r+1)/(n+1) where r is the number of estimators that exceed the unpermuted estimator in value and n is the total number of permutations. |



## Questions

When answering the assignment questions, please provide citations for statements derived from external sources. You can receive marks even if your answer does not match the answer key, provided that your rationale is supported by valid sources. Any citation style is acceptable.

1. Review the phenotype matrix: What was the sample size of each outbreak?

2. Review the genotype matrix generated by `Roary`: How many genes were tested for association? *Note: The pan-genome constructed by Roary is non-deterministic and therefore the number of genes can vary between runs. Any answer within a reasonable range (±100) is acceptable.*

3. Review the `Scoary` outputs: How many genes were found significantly associated with genomes from Outbreak 2?

4. Review the `Scoary` outputs: Genomes of which outbreak(s) encode the bacteriocin, Colicin?

5. Are colicin genes typically found in bacterial chromosomes or extrachromosomal DNA?

6. Do some research on the genes repeatedly found significantly associated to the different *Salmonella* outbreaks. What do the significant genes have in common?

7. What was one major shortcoming of this analysis? What might you do to address the problem?

## Closing Remarks

Congratulations, you have reached the end of the assignment! An important note to highlight is that in this assignment, you have only been presented one of the many methods to conduct microbial GWAS. Numerous alternative methods exist such as statistical modeling and deep learning, with each of their own advantages and disadvantages. For those interested, we leave you with the publication by [John Lees et al. (2020)](https://journals.asm.org/doi/full/10.1128/mBio.01344-20) who compared and contrasted different strategies to identify genetic association in microbial organisms.

## References

1. Bekal S, Berry C, Reimer AR, Van Domselaar G, Beaudry G, Fournier E, et al. Usefulness of High-Quality Core Genome Single-Nucleotide Variant Analysis for Subtyping the Highly Clonal and the Most Prevalent Salmonella enterica Serovar Heidelberg Clone in the Context of Outbreak Investigations. J Clin Microbiol. 2016 Feb;54(2):289–95.
2. Page AJ, Cummins CA, Hunt M, Wong VK, Reuter S, Holden MTG, et al. Roary: rapid large-scale prokaryote pan genome analysis. Bioinformatics. 2015 Nov 15;31(22):3691–3.
3. Seemann T. Prokka: rapid prokaryotic genome annotation. Bioinformatics. 2014 Jul 15;30(14):2068–9.
4. Brynildsrud O, Bohlin J, Scheffer L, Eldholm V. Rapid scoring of genes in microbial pan-genome-wide association studies with Scoary. Genome Biol. 2016 Nov 25;17(1):238.
5. Lees JA, Mai TT, Galardini M, Wheeler NE, Horsfield ST, Parkhill J, et al. Improved prediction of bacterial genotype-phenotype associations using interpretable pangenome-spanning regressions. MBio [Internet]. 2020 Jul 7;11(4). Available from: https://journals.asm.org/doi/10.1128/mBio.01344-20